# SQL Q&A
https://python.langchain.com/docs/tutorials/sql_qa/#agents

In [38]:
# Make sure to set your env varaiable
# export LANGSMITH_API_KEY=[API KEY]

from getpass import getpass
import os

if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass()
    os.environ["LANGSMITH_API_KEY"] = 'true'
    


In [39]:
from langchain_community.utilities import SQLDatabase

from toelo.player_elo.database_connection import get_connection_string, DATABASE_CONFIG


db_uri = get_connection_string(DATABASE_CONFIG)
db = SQLDatabase.from_uri(db_uri)
db.run("SELECT * FROM players_elo LIMIT 5;")

"[(33042, 2014, 'Migjen', 'Basha', 'Migjen Basha', 'migjen-basha', 'Switzerland', datetime.date(1987, 1, 5), 2593.2273077268505), (159029, 2013, 'Hüseyin', 'Kala', 'Hüseyin Kala', 'huseyin-kala', 'Turkey', datetime.date(1987, 5, 5), 2536.2046353257374), (167061, 2012, 'Leighton', 'McIntosh', 'Leighton McIntosh', 'leighton-mcintosh', 'Scotland', datetime.date(1993, 2, 6), 2255.7503253540335), (22598, 2014, 'Gianni', 'Munari', 'Gianni Munari', 'gianni-munari', 'Italy', datetime.date(1983, 6, 24), 2574.7604517201576), (118056, 2014, 'Mavroudis', 'Bougaidis', 'Mavroudis Bougaidis', 'mavroudis-bougaidis', 'Greece', datetime.date(1993, 6, 1), 2507.888919411905)]"

In [40]:
from pathlib import Path
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


from langchain_ollama import ChatOllama

# Chat model
# llm_path = Path.home() / Path("models") / "mistral-7b-openorca.Q4_0.gguf"
llm = ChatOllama(model="mistral:7b", n_threads=8)

# https://python.langchain.com/docs/tutorials/sql_qa/#agents

In [41]:
# Provide some instruction
from langchain_core.prompts import ChatPromptTemplate

system_message = system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
    )

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}


In [42]:
# Populate parameters
from typing_extensions import Annotated

class QueryOutput(TypedDict):
    """Generated SQL query

    Args:
        TypedDict (_type_): _description_
    """
    query: Annotated[str, ...,  "Syntatically valid SQL Query"]
    
def write_query(state: State):
    """Generate SQL query to fetch info

    Args:
        state (State): _description_
    """
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"]
        }
    )
    # print(prompt)
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}



In [43]:
# print(db.dialect)
# print(db.get_table_info())
# print(state["question"])

In [45]:
write_query({"question": "What is the highest elo from players_elo table?"})
write_query({"question" : "How many players exists in our database, you can use Players table"})

{'query': 'SELECT COUNT(*) FROM Players;'}